In [1]:
import pandas as pd
from geopy.distance import geodesic
from scipy.spatial import cKDTree
from tqdm import tqdm

# Wczytanie danych UFO
ufo_data = pd.read_csv("C:/Repos/ufo_project/data/final_file/cleared_data.csv")

# Wczytanie danych lotnisk
airports = pd.read_csv("C:/Repos/ufo_project/data/final_file/airports_UTF-8.csv", delimiter=";")

# Filtruj tylko lotniska typu 'large_airport'
airports_large = airports[airports['type'] == 'large_airport'].dropna(subset=['latitude_deg', 'longitude_deg', 'ident', 'name'])

# Przygotowanie danych lotnisk
airports_coords = airports_large[['latitude_deg', 'longitude_deg']].values
airport_tree = cKDTree(airports_coords)  # Budowanie drzewa k-D

# Funkcja do obliczania odległości
def find_nearest_airport(ufo_lat, ufo_lon):
    """
    Znajdź najbliższe lotnisko i oblicz odległość w kilometrach.
    """
    distance, index = airport_tree.query([ufo_lat, ufo_lon])
    nearest_airport = airports_large.iloc[index]
    airport_coords = (nearest_airport['latitude_deg'], nearest_airport['longitude_deg'])
    ufo_coords = (ufo_lat, ufo_lon)
    distance_km = geodesic(ufo_coords, airport_coords).kilometers  # Obliczenie odległości w km
    return nearest_airport['ident'], nearest_airport['name'], airport_coords, distance_km

# Przygotowanie list wynikowych
ufo_ids = []
event_names = []
cities = []
ufo_coords = []
distances = []
airport_keys = []
airport_names = []
airport_coords_list = []

# Przetwarzanie danych UFO
for _, ufo_event in tqdm(ufo_data.iterrows(), total=ufo_data.shape[0], desc="Przetwarzanie UFO zdarzenia"):
    try:
        ufo_lat = ufo_event['latitude']
        ufo_lon = ufo_event['longitude']
        if pd.isna(ufo_lat) or pd.isna(ufo_lon):
            continue
        # Znajdź najbliższe lotnisko
        airport_key, airport_name, airport_coord, distance = find_nearest_airport(ufo_lat, ufo_lon)

        # Dodaj wyniki do list
        ufo_ids.append(ufo_event['id'])
        event_names.append(ufo_event.get('event_name', 'N/A'))  # Zmień na nazwę kolumny zdarzenia, jeśli istnieje
        cities.append(ufo_event.get('city', 'Unknown'))
        ufo_coords.append(f"{ufo_lat}, {ufo_lon}")
        distances.append(distance)
        airport_keys.append(airport_key)
        airport_names.append(airport_name)
        airport_coords_list.append(f"{airport_coord[0]}, {airport_coord[1]}")

    except Exception as e:
        print(f"Błąd przetwarzania zdarzenia ID {ufo_event['id']}: {e}")

# Tworzenie tabeli wynikowej
result_df = pd.DataFrame({
    'id': ufo_ids,
    'zdarzenie': event_names,
    'miasto': cities,
    'współrzędne_zdarzenia': ufo_coords,
    'odległość_km': distances,
    'klucz_lotniska': airport_keys,
    'nazwa_lotniska': airport_names,
    'współrzędne_lotniska': airport_coords_list
})

# Zapisanie tabeli wynikowej do pliku
output_path = "C:/Repos/ufo_project/data/final_file/ufo_airport_distances.csv"
result_df.to_csv(output_path, index=False)

print(f"Zakończono przetwarzanie. Wyniki zapisano do pliku: {output_path}")


Przetwarzanie UFO zdarzenia: 100%|███████████████████████████████████████████████████████████████| 84057/84057 [00:28<00:00, 2925.19it/s]


Zakończono przetwarzanie. Wyniki zapisano do pliku: C:/Repos/ufo_project/data/final_file/ufo_airport_distances.csv


In [2]:
git pull origin main

SyntaxError: invalid syntax (4098769945.py, line 1)